In [ ]:
%pip install langchain_openai
%pip install langchain
%pip install python-docx

In [8]:
import os
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

Set API Keys from .env file (example provided in github)

In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
llm = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
terms_prompt_template = PromptTemplate.from_template("List {num_terms} key terms to learn about '{topic}'.")
definition_prompt_template = PromptTemplate.from_template("Provide a definition for the term '{term}':")

In [ ]:
# Generates n terms for a given topic (if no n provided, then 10 terms created by default)
def get_terms_for_topic(topic, num_terms=10):
        prompt = terms_prompt_template.format(topic=topic, num_terms=num_terms)
        response = llm(prompt).text.strip()
        terms = response.split('\n')
        return terms

In [ ]:
# Generate flashcards
def generate_flashcards(topic, num_terms=10):
        terms = get_terms_for_topic(topic, num_terms)
        flashcards = []

        for term in terms:
            prompt = definition_prompt_template.format(term=term)
            response = llm(prompt).text.strip()
            flashcards.append({"Word": term, "Definition": response})
        
        return flashcards

In [ ]:
# Supported formats for files in Quizlet: Supported file types: .docx, .pdf, .pptx
from docx import Document
def save_to_docx(flashcards, filename="flashcards.docx"):
        doc = Document()
        doc.add_heading('Flashcards', 0)

        for flashcard in flashcards:
            term = flashcard["Word"]
            definition = flashcard["Definition"]
            
            doc.add_paragraph(term + ': ' + definition)

        doc.save(filename)

In [ ]:
def chatbot():
    print("Welcome to the Flashcards Chatbot!")
    topic = input("Please enter a topic you want to study").strip().lower()
    num_terms = input("Please enter how many terms you want to study")

    flashcards = generate_flashcards(topic, num_terms)
    save_to_docx(flashcards)

In [ ]:
if __name__ == "__main__":
    chatbot()